# openai

In [1]:
# https://gym.openai.com/
# https://gym.openai.com/docs/
try:
    import gym
    
except ModuleNotFoundError as e:
    import sys
    !{sys.executable} -m pip install gym
    import gym
finally:
    from gym.envs.registration import register


# 1. Deteministic 환경.

## 게임 환경설정

In [2]:
# Gym 환경설정
# https://gym.openai.com/envs/ 에 들어가면 다양한 환경을 살펴볼 수 있음
# Frozen-Lake : https://gym.openai.com/envs/FrozenLake-v0/ 
register(
    id = 'FrozenLake-v1', 
    entry_point='gym.envs.toy_text:FrozenLakeEnv', 
    kwargs={
        # 'is_slippery': False 이면 Deteministic 환경.
        # 'is_slippery': True 이면 Stochastic 환경.
        'map_name' : '4x4', # 4x4 크기의 맵
        'is_slippery' : False # 미끄러질 가능성 없음 (나중에 다룰 것)
    }
)

In [3]:
env = gym.make("FrozenLake-v1")
env.render()


SFFF
FHFH
FFFH
HFFG


## 기본 사용법 익히기

In [4]:
env.render()


SFFF
FHFH
FFFH
HFFG


In [5]:
env.observation_space, env.action_space, env.observation_space.n, env.action_space.n

(Discrete(16), Discrete(4), 16, 4)

In [6]:
# LEFT = 0
# DOWN = 1
# RIGHT = 2
# UP = 3

# 임의로 방향 하나를 선택함.
action = env.action_space.sample()
action

0

In [7]:
observation, reward, done, info = env.step(action)
env.render()

  (Left)
SFFF
FHFH
FFFH
HFFG


In [8]:
env.reset()

0

In [9]:
action = env.action_space.sample()
observation, reward, done, info = env.step(0)
env.render()

  (Left)
SFFF
FHFH
FFFH
HFFG


In [10]:
print("위치: ", observation)
print("보상: ",reward)
print("Game Over?: ",done)
print("정보(확률): ", info) # 내가 의도한대로 갈 확률
print("행동: ", action)

위치:  0
보상:  0.0
Game Over?:  False
정보(확률):  {'prob': 1.0}
행동:  3


In [11]:
# 초기화
env.reset()

0

In [12]:
env.render()


SFFF
FHFH
FFFH
HFFG


## Q 러닝이란


Q-learning에서는 앞으로 어떤 action을 취할지 결정하기 위해서 Q-table을 사용하는데,
Q-table은 맵과 같은 모양(Frozen-Lake의 경우 4x4)으로 만들어지며
각 칸에는 해당 칸에서 각각의 action을 취할 때 기대하는 reward 값으로 채워진다.
(한 state에서 취할 수 있는 action이 상하좌우 4개이기 때문에 각 칸마다 4개의 값이 필요하다)

Q-learning을 진행하면서 Q-table의 각 reward 값을 업데이트 하여 최상의 결과값을 리턴할 수 있도록 해야한다.

Dummy Q-learning은 0으로 채워놓은 Q-테이블에서 시작해서 각 state에서 취할 action을 랜덤으로 선택한다.
(상하좌우 모두 reward가 0이기 때문에)

Frozen-Lake에서는 결승점에 도달할 때만 reward 1이 주어지기 때문에 결승점에 도착하기 전에는
reward는 무조건 0이 리턴된다

action은 랜덤으로 정해지기 때문에 보통 구멍에 빠져서 게임이 종료되는 상황이 
반복 되다가 우연히 결승점에 다다르면 그제서야 reward가 1이 주어진다. 

그럼 학습하는 agent는 이 전 칸에서 어떤 action(예를 들어 14번칸에서 오른쪽)을 하면 결승점에 다다른다는 것을
알게 된다. 그럼 이 전 칸의 Q-table을 업데이트 할 수 있다(14번칸에서 오른쪽의 값을 1로 업데이트).

이처럼 Q-table은 우연히 결승점에 도착한 상황을 토대로 시작점까지 거꾸로 거슬러 올라가면서 Q값을 업데이트하는
방식을 사용한다. (결승점에 도착하기 위해서는 14번에서 오른쪽으로 가야한다. 14번에 다다르기 위해서는 
10번에서 아래로 가야한다. 10번에 가기 위해서는 ... ) 이런 알고리즘을 식으로 표현하면 아래와 같다.

Q[state, action] = reward + max(Q[new_state])

<< Dummy Q-learning >>  
2강의 Dummy Q-learning에서는 1이 채워지지 않는 칸에서 action이 랜덤으로 정해지기 때문에 
갈림길의 경우에는 랜덤으로 돌아다니다가 어떤 길로 우연히 처음 들게 되었는지에 따라 Q-테이블의 모양이 달라지게 된다.
한 state의 여러 action 중에서 1 값을 갖는 건 무조건 하나일 수 밖에 없다.
일단 1이 채워지면 그 다음부터는 max값을 따라서 1이 채워진 방향으로만 지나갈 것이기 때문이다.
또한, 아무리 빠른 지름길이 있더라도, 1이 채워지지 않게 되면 그 길로 들어서지 못하게 되기 때문에 성공하더라도 최단거리일 가능성은 낮다.

In [13]:
import numpy as np      ####  http://yujuwon.tistory.com/entry/NumPy
import random

## 순서
### 1. Q table 만들기
### 2. 게임을 진행하며 Q 테이블 업데이트 하기
### 3. Q테이블로 테스트하기

#### page 163

### 1. Q table 만들기

In [14]:
# np.zeros((16, 4))
Q = np.zeros((env.observation_space.n, env.action_space.n))    # (16,4) : 4*4 map + 상하좌우 4개
Q

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

#### page.164

### 2. 학습시키며 Q table 업데이트 하기

In [15]:
# 횟수 초기화
num_episodes = 1000
learning_rate = 0.3
discount_rate = 0.99

In [16]:
total_reward = 0.0
for epi in range(num_episodes):
    observation = env.reset() # 초기화
    
    episode_reward = 0.0
    
    for t in range(100):
        # 1턴 실행 후의 위치
        current_state = observation
        
        # 학습을 빨리 하기 위해서 넣는 코드
        # 없으면 계속 LEFT로 가기 때문
        # 전체가 0이면 처리
        if not Q[current_state, :].any():
            action = env.action_space.sample()
        elif np.random.rand() < 0.5:
            # 랜덤 선택
            action = env.action_space.sample()
        else:
            # 최대값
            action = np.argmax(Q[current_state])
        
        observation, reward, done, info = env.step(action)
        
        # 성공 사례를 만들어야 측정이 가능함.
        # 현 위치의 최대값 = 성공 가능성
        # 이전 위치의 현 방향 선택 = 기존의 성공 가능성
        Q[current_state, action] += learning_rate * (reward + discount_rate * np.max(Q[observation, :]) - Q[current_state, action]) 
        
        if done:
            episode_reward +=reward
            break
    if epi % 500 == 0:
        print(epi, "번 진행중..")
    total_reward += episode_reward
            
    

0 번 진행중..
500 번 진행중..


In [17]:
total_reward/ num_episodes

0.265

In [18]:
# 학습이 끝난 Q값
Q

array([[0.94148015, 0.95099005, 0.94387816, 0.94148015],
       [0.94148015, 0.        , 0.95741776, 0.93558652],
       [0.61566823, 0.97004881, 0.4838126 , 0.91110315],
       [0.6930384 , 0.        , 0.21609863, 0.17976737],
       [0.95099005, 0.96059601, 0.        , 0.94148015],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.98008768, 0.        , 0.8549699 ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.96059601, 0.        , 0.970299  , 0.95099005],
       [0.96059601, 0.9801    , 0.9801    , 0.        ],
       [0.97011416, 0.99      , 0.        , 0.96541822],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.9800999 , 0.99      , 0.970299  ],
       [0.98009994, 0.98999995, 1.        , 0.98009998],
       [0.        , 0.        , 0.        , 0.        ]])

### 3. Q테이블로 테스트하기

#### p.166

In [19]:
# 횟수 초기화
num_episodes = 1000

In [20]:
total_reward = 0.0
for i_episode in range(num_episodes):
    observation = env.reset()
    
    episode_reward = 0.0
    for t in range(100):
        #1턴 후의 위치
        current_state = observation
        
        # Q값이 최대가 되는 행동을 선택함
        action = np.argmax(Q[current_state])
        
        # 1턴 실행
        observation, reward, done, info = env.step(action)
        
        #종료
        if done:
            episode_reward += reward
            
    # 총 reward
    total_reward += episode_reward
        

In [365]:
print(total_reward)

1000.0


In [366]:

total_reward / num_episodes

1.0

# 2. Stochastic 환경.

In [43]:
# Gym 환경설정
# https://gym.openai.com/envs/ 에 들어가면 다양한 환경을 살펴볼 수 있음
# Frozen-Lake : https://gym.openai.com/envs/FrozenLake-v0/ 
register(
    id = 'FrozenLake-v2', 
    entry_point='gym.envs.toy_text:FrozenLakeEnv', 
    kwargs={
        # 'is_slippery': False 이면 Deteministic 환경.
        # 'is_slippery': True 이면 Stochastic 환경.
        'map_name' : '4x4', # 4x4 크기의 맵
        'is_slippery' : True # 미끄러질 가능성 없음 (나중에 다룰 것)
    }
)

Error: Cannot re-register id: FrozenLake-v2

In [44]:
env = gym.make("FrozenLake-v2")
env.render()


SFFF
FHFH
FFFH
HFFG


### 사용설명

In [45]:
env.reset()

0

In [46]:
action = env.action_space.sample()
observation, reward, done, info = env.step(action)
env.render()

  (Right)
SFFF
FHFH
FFFH
HFFG


In [47]:
print("위치: ", observation)
print("보상: ",reward)
print("Game Over?: ",done)
print("정보(확률): ", info) # 내가 의도한대로 갈 확률
print("행동: ", action)

위치:  4
보상:  0.0
Game Over?:  False
정보(확률):  {'prob': 0.3333333333333333}
행동:  2


## 순서
### 1. Q table 만들기
### 2. 게임을 진행하며 Q 테이블 업데이트 하기
### 3. Q테이블로 테스트하기

#### page 163

### 1. Q table 만들기

In [48]:
import numpy as np
# np.zeros((16, 4))
Q = np.zeros((env.observation_space.n, env.action_space.n))    # (16,4) : 4*4 map + 상하좌우 4개
Q

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

#### page.164

### 2. 학습시키며 Q table 업데이트 하기

In [27]:
# 횟수 초기화
num_episodes = 300000
learning_rate = 0.3
discount_rate = 0.99

In [28]:
total_reward = 0.0
recent_reward = []
recent_size = 100

for epi in range(num_episodes):
    observation = env.reset() # 초기화
    
    episode_reward = 0.0
    
    for t in range(100):
        # 1턴 실행 후의 위치
        current_state = observation
        
        # 학습을 빨리 하기 위해서 넣는 코드
        # 없으면 계속 LEFT로 가기 때문
        # 전체가 0이면 처리
        
        # if not Q[current_state, :].any():
        # action = env.action_space.sample()
        if np.random.rand() < 0.1: # 변경 된걸 확인하세요.
            # 랜덤 선택
            action = env.action_space.sample()
        else:
            # 최대값
            action = np.argmax(Q[current_state])
        
        observation, reward, done, info = env.step(action)
        
        # 성공 사례를 만들어야 측정이 가능함.
        # 현 위치의 최대값 = 성공 가능성
        # 이전 위치의 현 방향 선택 = 기존의 성공 가능성
        Q[current_state, action] += learning_rate * (reward + discount_rate * np.max(Q[observation, :]) - Q[current_state, action]) 
        
        if done:
            episode_reward +=reward
            break
    if epi % 10000 == 0:
        print(epi, "번 진행중..")
    total_reward += episode_reward
    
    recent_reward.append(episode_reward)
    if len(recent_reward) >= recent_size:
        # 최근 100개에 대해서 검사
        recent_reward.pop(0)
        
        if sum(recent_reward) / recent_size > 0.7:
            print("0.7이 넘었습니다. 멈추겠습니다.")
            break

            
    

0 번 진행중..
10000 번 진행중..
20000 번 진행중..
30000 번 진행중..
40000 번 진행중..
50000 번 진행중..
60000 번 진행중..
70000 번 진행중..
80000 번 진행중..
90000 번 진행중..
100000 번 진행중..
110000 번 진행중..
120000 번 진행중..
130000 번 진행중..
140000 번 진행중..
150000 번 진행중..
160000 번 진행중..
170000 번 진행중..
180000 번 진행중..
190000 번 진행중..
200000 번 진행중..
210000 번 진행중..
220000 번 진행중..
230000 번 진행중..
240000 번 진행중..
250000 번 진행중..
260000 번 진행중..
270000 번 진행중..
280000 번 진행중..
290000 번 진행중..


In [29]:
total_reward/ num_episodes

0.3347

In [30]:
sum(recent_reward)/recent_size

0.29

In [31]:
# 학습이 끝난 Q값
Q

array([[0.57518427, 0.46972455, 0.51374281, 0.47221161],
       [0.0664377 , 0.23360794, 0.35288834, 0.46511191],
       [0.27605549, 0.34324715, 0.32093682, 0.40912368],
       [0.31749354, 0.15824233, 0.23682097, 0.37376393],
       [0.60826663, 0.36966638, 0.18275754, 0.45788789],
       [0.        , 0.        , 0.        , 0.        ],
       [0.1146231 , 0.13135162, 0.30698845, 0.00415065],
       [0.        , 0.        , 0.        , 0.        ],
       [0.55898479, 0.38577184, 0.13200961, 0.65689771],
       [0.56632572, 0.69057155, 0.45768803, 0.46736318],
       [0.39329225, 0.45478775, 0.17288841, 0.17863629],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.51893475, 0.5204799 , 0.86332762, 0.58054121],
       [0.72881053, 0.97853049, 0.71667287, 0.65124712],
       [0.        , 0.        , 0.        , 0.        ]])

### 3. Q테이블로 테스트하기

#### p.166

In [32]:
# 횟수 초기화
num_episodes = 1000

In [33]:
total_reward = 0.0
for i_episode in range(num_episodes):
    observation = env.reset()
    
    episode_reward = 0.0
    for t in range(100):
        #1턴 후의 위치
        current_state = observation
        
        # Q값이 최대가 되는 행동을 선택함
        action = np.argmax(Q[current_state])
        
        # 1턴 실행
        observation, reward, done, info = env.step(action)
        
        #종료
        if done:
            episode_reward += reward
            
    # 총 reward
    total_reward += episode_reward
        

In [34]:
print(total_reward)

591.0


In [35]:
total_reward / num_episodes

0.591

# DQN ( p.167)

In [36]:
import gym

import numpy as np

In [37]:
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F

# Stochastic 환경.
env = gym.make('FrozenLake-v2')

### 신경망 구성 p.169

In [62]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(16, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 96)
        self.fc4 = nn.Linear(96, 96)
        self.fc5 = nn.Linear(96, 64)
        self.fc6 = nn.Linear(64, 64)
        self.fc7 = nn.Linear(64, 4)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = F.relu(self.fc6(x))
        x = self.fc7(x)
        
        return x

In [63]:
model = Net()

### onehot2tensor, applymodel p. 170

In [64]:
def onehot2tensor(state):
    tmp = np.zeros(16)
    tmp[state] = 1
    vector = np.array(tmp, dtype='float32')
    tensor = torch.from_numpy(vector).float()
    return tensor

def applymodel(tensor):
    output_tensor = model(tensor)
    output_array = output_tensor.data.numpy()
    return output_tensor, output_array

In [136]:
for x in range(16):
    print(onehot2tensor(x))

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

## 최적화(학습) - (p.171)

In [65]:
# loss가 수렴하지 않음 (책)

In [140]:
total_reward = 0.0
num_episodes = 5000
discount_rate = 0.99

criterion = nn.MSELoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)

for i_episode in range(num_episodes):
    observation = env.reset()
    
    # 현재 게임 reward
    episode_reward = 0.0
    
    # 누적 오차
    total_loss = 0.0
    
    for t in range(100):
        current_state = observation
        
        # 경사 초기화
        optimizer.zero_grad()
        
        # 현위치 onehot encoding 
        current_tensor = onehot2tensor(current_state)

        if (i_episode+1) % 500 == 0:
            print(current_tensor)
            current_output_tensor, current_output_array = applymodel(current_tensor)
            print(current_output_tensor)
            # print("{}번: total_loss: {}, total_reward: {} ".format(i_episode+1, total_loss, total_reward))
            
            
        # 모형에 입력
        current_output_tensor, current_output_array = applymodel(current_tensor)
        # 행동 선택
        if np.random.rand() < 0.1:
            #무작위 선택
            action = env.action_space.sample()
        else:
            # Q값이 최대가 되도록
            action = np.argmax(current_output_array)

        ## 1턴 실행
        observation, reward, done, info = env.step(action)
        
        # onehot 벡터 텐서로 변환
        observation_tensor = onehot2tensor(observation)
        
        # 모형에 입력
        observation_output_tensor, observation_output_array = applymodel(observation_tensor)
        
        # Q값 업데이트
        # y값
        q = reward + discount_rate * np.max(observation_output_array) # reward는 마지막 value를 위해 더함
        
        # 기존 보드의 q값 복사
        q_array = np.copy(current_output_array) # 이전 값
        q_array[action] = q
        q_variable = torch.from_numpy(q_array)
        
        # 오차 계산 - q_array와 q
        # 현재 아웃풋 값과, 기존 q어레이
        # y값은 올바른 선택일 때의 최대값,
        # q^은 현재 선택의 결과
        
        # current_output_tensor: 현재 측정값
        # q_variable: 최고의 선택을 했을 때 y값
        loss = criterion(current_output_tensor, q_variable)
        
        # 역전파
        loss.backward()
        
        # 웨이트 업데이트
        optimizer.step()
        
        # 오차 누적 계산
        total_loss += loss.item()
        
        if done:
            # 종료
            episode_reward += reward
            
            break
    total_reward += episode_reward
    
    # 누적오차 및 보상률
    if (i_episode+1) % 500 == 0:
        print("{}번: total_loss: {}, total_reward: {} ".format(i_episode+1, total_loss, total_reward))
        
        

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0.0498, 0.0501, 0.0498, 0.0503], grad_fn=<AddBackward0>)
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0.0498, 0.0501, 0.0498, 0.0502], grad_fn=<AddBackward0>)
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0.0497, 0.0501, 0.0498, 0.0501], grad_fn=<AddBackward0>)
tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0.0497, 0.0500, 0.0497, 0.0501], grad_fn=<AddBackward0>)
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0.0496, 0.0500, 0.0497, 0.0500], grad_fn=<AddBackward0>)
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0.0496, 0.0500, 0.0497, 0.0499], grad_fn=<AddBackward0>)
tensor([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0.0496, 0.0499, 0.0497, 0.0499], grad_fn=<AddBackward0>)
500번: total_loss: 4.39098137405835

tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0.5517, 0.5828, 0.5697, 0.5387], grad_fn=<AddBackward0>)
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0.5517, 0.5832, 0.5703, 0.5387], grad_fn=<AddBackward0>)
tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0.5517, 0.5836, 0.5708, 0.5387], grad_fn=<AddBackward0>)
tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0.5517, 0.5840, 0.5713, 0.5387], grad_fn=<AddBackward0>)
tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0.5517, 0.5843, 0.5717, 0.5388], grad_fn=<AddBackward0>)
tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0.5517, 0.5846, 0.5720, 0.5388], grad_fn=<AddBackward0>)
tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0.5517, 0.5848, 0.5724, 0.5388], grad_fn=<AddBackward0>)
4000번: total_loss: 6.1989472669665

### DQL 테스트

In [143]:
total_reward = 0.0
num_episodes = 1000

for i_episode in range(num_episodes):
    observation = env.reset()
    
    # 현재 게임 reward
    episode_reward = 0.0
    
    
    for t in range(100):
        current_state = observation
        
        
        # 현위치 onehot encoding 
        current_tensor = onehot2tensor(current_state)
        # 모형에 입력
        current_output_tensor, current_output_array = applymodel(current_tensor)
        
        # Q테이블에서 제일 큰거 선택
        action = np.argmax(current_output_array)

        ## 1턴 실행
        observation, reward, done, info = env.step(action)
        
        if done:
            # 종료
            episode_reward += reward
            break
            
    total_reward += episode_reward
    
    # 누적오차 및 보상률
    if (i_episode+1) % 500 == 0:
        print("{}번:".format(i_episode+1))
        print(total_reward)
        print("-"*30)
      

500번:
24.0
------------------------------
1000번:
53.0
------------------------------


In [144]:
print(total_reward)  
print(total_reward/1000)  

53.0
0.053
